pytorch関連のインストール

In [3]:
!pip3 install torch torchvision torchtext

    100% |████████████████████████████████| 484.0MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b092000 @  0x7fe4357f51c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 18.6MB/s 
    100% |████████████████████████████████| 51kB 17.8MB/s 
    100% |████████████████████████████████| 2.0MB 13.4MB/s 
    100% |████████████████████████████████| 51kB 13.3MB/s 
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext


  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [4]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Preparing to unpack .../04-gnupg_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg (2.1.15-1ubuntu8.1) over (2.1.15-1ubuntu8) ...
Preparing to unpack .../05-gnupg-agent_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking gnupg-agent (2.1.15-1ubuntu8

Setting up libgirepository-1.0-1:amd64 (1.54.1-1) ...
Setting up libkmod2:amd64 (24-1ubuntu2) ...
Setting up gir1.2-glib-2.0:amd64 (1.54.1-1) ...
Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up gnupg-agent (2.1.15-1ubuntu8.1) ...
Setting up dirmngr (2.1.15-1ubuntu8.1) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc

In [5]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
path = "drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch"
os.chdir(path)

In [8]:
!pwd

/content/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch


In [0]:
import os
import sys
import time
import glob

import torch
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets

# from model_bowman16 import SNLIClassifier
from model import SNLIClassifier
from util import get_args, makedirs

In [10]:
sys.argv = [sys.argv[0],
            '--epochs', '20', '--d_embed', '300',
            '--d_proj', '300', 
            # '--dp_ratio', '0.5', 
            # '--n_layers', '2',
            '--d_hidden', '100', 
            '--train_embed',
            # '--no-bidirectional', 
            # '--no-projection',
            '--word_vectors', 'glove.6B.300d', 
            '--vector_cache', '../input/vector_cache']
args = get_args()
args
# !python train.py --epochs 5 --d_embed 300 --gpu -1 --word_vectors glove.6B.300d

Namespace(batch_size=128, birnn=True, d_embed=300, d_hidden=100, d_proj=300, dev_every=1000, dp_ratio=0.2, epochs=20, fix_emb=False, gpu=0, log_every=50, lower=True, lr=0.001, n_layers=1, projection=True, resume_snapshot='', save_every=1000, save_path='results', vector_cache='../input/vector_cache', word_vectors='glove.6B.300d')

In [0]:
torch.cuda.set_device(args.gpu)

In [0]:
inputs = data.Field(lower=args.lower)
answers = data.Field(sequential=False, unk_token=None)
train, dev, test = datasets.SNLI.splits(inputs, answers, root="../input")

In [0]:
inputs.build_vocab(train, dev, test)

inputs.vocab.load_vectors(args.word_vectors, cache=args.vector_cache)
# if args.word_vectors:
#     if os.path.isfile(args.vector_cache):
#         inputs.vocab.vectors = torch.load(args.vector_cache)
#     else:
#         inputs.vocab.load_vectors(args.word_vectors)
#         makedirs(os.path.dirname(args.vector_cache))
#         torch.save(inputs.vocab.vectors, args.vector_cache)

answers.build_vocab(train)

In [0]:
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
            (train, dev, test), batch_size=args.batch_size, device=args.gpu)

In [15]:
answers.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'contradiction': 1, 'entailment': 0, 'neutral': 2})

In [0]:
config = args
config.n_embed = len(inputs.vocab)
config.d_out = len(answers.vocab)
config.n_cells = config.n_layers

# double the number of cells for bidirectional networks
if config.birnn:
    config.n_cells *= 2
    
criterion = nn.CrossEntropyLoss()

In [0]:
if args.resume_snapshot:
    model = torch.load(args.resume_snapshot, map_location=lambda storage, locatoin: storage.cuda(args.gpu))
else:
    model = SNLIClassifier(config)
    if args.word_vectors:
        model.embed.weight.data.copy_(inputs.vocab.vectors)
        model.cuda(args.gpu)

opt = O.Adam(model.parameters(), lr=args.lr, weight_decay=1e-6)
# opt = O.RMSprop(model.parameters(), lr=args.lr, weight_decay=1e-6)

In [29]:
iterations = 0
start = time.time()
best_dev_acc = -1
train_iter.repeat = False
header = '  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy'
dev_log_template = ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{:8.6f},{:12.4f},{:12.4f}'.split(','))
log_template =     ' '.join('{:>6.0f},{:>5.0f},{:>9.0f},{:>5.0f}/{:<5.0f} {:>7.0f}%,{:>8.6f},{},{:12.4f},{}'.split(','))
makedirs(args.save_path)
print(header)

for epoch in range(args.epochs):
    train_iter.init_epoch()
    n_correct, n_total = 0, 0
    for batch_idx, batch in enumerate(train_iter):

        # switch model to training mode, clear gradient accumulators
        model.train(); opt.zero_grad()

        iterations += 1

        # forward pass
        answer = model(batch)

        # calculate accuracy of predictions in the current batch
        n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
        n_total += batch.batch_size
        train_acc = 100. * n_correct/n_total

        # calculate loss of the network output with respect to training labels
        loss = criterion(answer, batch.label)

        # backpropagate and update optimizer learning rate
        loss.backward(); opt.step()

        # checkpoint model periodically
        if iterations % args.save_every == 0:
            snapshot_prefix = os.path.join(args.save_path, 'snapshot')
            snapshot_path = snapshot_prefix + '_acc_{:.4f}_loss_{:.6f}_iter_{}_model.pt'.format(train_acc, loss.item(), iterations)
            torch.save(model, snapshot_path)
            for f in glob.glob(snapshot_prefix + '*'):
                if f != snapshot_path:
                    os.remove(f)

        # evaluate performance on validation set periodically
        if iterations % args.dev_every == 0:

            # switch model to evaluation mode
            model.eval(); dev_iter.init_epoch()

            # calculate accuracy on validation set
            n_dev_correct, dev_loss = 0, 0
            with torch.no_grad():
                for dev_batch_idx, dev_batch in enumerate(dev_iter):
                     answer = model(dev_batch)
                     n_dev_correct += (torch.max(answer, 1)[1].view(dev_batch.label.size()) == dev_batch.label).sum().item()
                     dev_loss = criterion(answer, dev_batch.label)
            dev_acc = 100. * n_dev_correct / len(dev)

            print(dev_log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), dev_loss.item(), train_acc, dev_acc))

            # update best valiation set accuracy
            if dev_acc > best_dev_acc:

                # found a model with better validation set accuracy

                best_dev_acc = dev_acc
                snapshot_prefix = os.path.join(args.save_path, 'best_snapshot')
                snapshot_path = snapshot_prefix + '_devacc_{}_devloss_{}__iter_{}_model.pt'.format(dev_acc, dev_loss.item(), iterations)

                # save model, delete previous 'best_snapshot' files
                torch.save(model, snapshot_path)
                for f in glob.glob(snapshot_prefix + '*'):
                    if f != snapshot_path:
                        os.remove(f)

        elif iterations % args.log_every == 0:

            # print progress message
            print(log_template.format(time.time()-start,
                epoch, iterations, 1+batch_idx, len(train_iter),
                100. * (1+batch_idx) / len(train_iter), loss.item(), ' '*8, n_correct/n_total*100, ' '*12))




  Time Epoch Iteration Progress    (%Epoch)   Loss   Dev/Loss     Accuracy  Dev/Accuracy
     6     0        50    50/4292        1% 1.030925               38.8906             
     9     0       100   100/4292        2% 1.090853               43.0703             
    11     0       150   150/4292        3% 1.016042               45.6615             
    14     0       200   200/4292        5% 0.866493               47.9688             
    16     0       250   250/4292        6% 1.011518               49.7375             
    19     0       300   300/4292        7% 0.887961               51.2630             
    22     0       350   350/4292        8% 0.841663               52.4911             
    24     0       400   400/4292        9% 0.798525               53.6836             
    27     0       450   450/4292       10% 0.769812               54.6562             
    29     0       500   500/4292       12% 0.818466               55.3297             
    32     0       550   550/42

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


    69     0      1000  1000/4292       23% 0.672467 0.948961      59.8211      66.7141
    83     0      1050  1050/4292       24% 0.774817               60.1496             
    86     0      1100  1100/4292       26% 0.832548               60.3857             
    89     0      1150  1150/4292       27% 0.838880               60.6440             
    91     0      1200  1200/4292       28% 0.818148               60.8958             
    94     0      1250  1250/4292       29% 0.780609               61.1569             
    96     0      1300  1300/4292       30% 0.698543               61.4050             
    99     0      1350  1350/4292       31% 0.654172               61.6632             
   101     0      1400  1400/4292       33% 0.735562               61.8666             
   104     0      1450  1450/4292       34% 0.771493               62.0652             
   107     0      1500  1500/4292       35% 0.791026               62.2714             
   109     0      1550  1550/429

   252     0      3350  3350/4292       78% 0.690035               66.6301             
   255     0      3400  3400/4292       79% 0.701435               66.7022             
   258     0      3450  3450/4292       80% 0.651976               66.7781             
   260     0      3500  3500/4292       82% 0.740778               66.8379             
   263     0      3550  3550/4292       83% 0.653278               66.9069             
   265     0      3600  3600/4292       84% 0.659171               66.9690             
   268     0      3650  3650/4292       85% 0.680123               67.0298             
   271     0      3700  3700/4292       86% 0.791489               67.0914             
   273     0      3750  3750/4292       87% 0.703960               67.1733             
   276     0      3800  3800/4292       89% 0.526261               67.2385             
   278     0      3850  3850/4292       90% 0.651562               67.3072             
   281     0      3900  3900/429

   416     1      5700  1408/4292       33% 0.602217               75.8323             
   418     1      5750  1458/4292       34% 0.694334               75.8257             
   421     1      5800  1508/4292       35% 0.545082               75.8315             
   423     1      5850  1558/4292       36% 0.561304               75.8454             
   426     1      5900  1608/4292       37% 0.568920               75.8323             
   429     1      5950  1658/4292       39% 0.492721               75.8213             
   445     1      6000  1708/4292       40% 0.531965 0.850936      75.8146      73.7249
   459     1      6050  1758/4292       41% 0.805735               75.8097             
   462     1      6100  1808/4292       42% 0.598084               75.8003             
   464     1      6150  1858/4292       43% 0.664587               75.7981             
   467     1      6200  1908/4292       44% 0.627258               75.8075             
   469     1      6250  1958/429

...

  5172    17     76200  3236/4292       75% 0.342777               91.4690             
  5175    17     76250  3286/4292       77% 0.325718               91.4383             
  5177    17     76300  3336/4292       78% 0.283087               91.4142             
  5180    17     76350  3386/4292       79% 0.239798               91.4028             
  5182    17     76400  3436/4292       80% 0.305071               91.3719             
  5185    17     76450  3486/4292       81% 0.299090               91.3540             
  5188    17     76500  3536/4292       82% 0.256941               91.3382             
  5190    17     76550  3586/4292       84% 0.253486               91.3202             
  5193    17     76600  3636/4292       85% 0.229467               91.2956             
  5195    17     76650  3686/4292       86% 0.301011               91.2806             
  5198    17     76700  3736/4292       87% 0.270324               91.2636             
  5201    17     76750  3786/429

  5325    18     78550  1294/4292       30% 0.193569               92.5123             
  5328    18     78600  1344/4292       31% 0.218492               92.4880             
  5330    18     78650  1394/4292       32% 0.269266               92.4761             
  5333    18     78700  1444/4292       34% 0.260286               92.4521             
  5336    18     78750  1494/4292       35% 0.189931               92.4317             
  5338    18     78800  1544/4292       36% 0.183833               92.4066             
  5341    18     78850  1594/4292       37% 0.196614               92.3958             
  5343    18     78900  1644/4292       38% 0.263041               92.3909             
  5346    18     78950  1694/4292       39% 0.189143               92.3701             
  5362    18     79000  1744/4292       41% 0.236860 0.938239      92.3627      74.9848
  5365    18     79050  1794/4292       42% 0.214307               92.3425             
  5367    18     79100  1844/429

  5475    18     80900  3644/4292       85% 0.291292               91.5707             
  5477    18     80950  3694/4292       86% 0.279685               91.5581             
  5494    18     81000  3744/4292       87% 0.234091 0.895491      91.5348      74.9644
  5497    18     81050  3794/4292       88% 0.321973               91.5166             
  5499    18     81100  3844/4292       90% 0.208262               91.4971             
  5502    18     81150  3894/4292       91% 0.334946               91.4771             
  5504    18     81200  3944/4292       92% 0.302493               91.4556             
  5507    18     81250  3994/4292       93% 0.298219               91.4293             
  5510    18     81300  4044/4292       94% 0.432073               91.3993             
  5512    18     81350  4094/4292       95% 0.252145               91.3814             
  5515    18     81400  4144/4292       97% 0.478343               91.3633             
  5517    18     81450  4194/429

  5641    19     83250  1702/4292       40% 0.193617               92.6135             
  5643    19     83300  1752/4292       41% 0.195356               92.5902             
  5646    19     83350  1802/4292       42% 0.217477               92.5807             
  5648    19     83400  1852/4292       43% 0.255846               92.5634             
  5651    19     83450  1902/4292       44% 0.183437               92.5502             
  5653    19     83500  1952/4292       45% 0.174152               92.5309             
  5656    19     83550  2002/4292       47% 0.107278               92.5219             
  5659    19     83600  2052/4292       48% 0.221567               92.4852             
  5661    19     83650  2102/4292       49% 0.216217               92.4696             
  5664    19     83700  2152/4292       50% 0.285426               92.4511             
  5666    19     83750  2202/4292       51% 0.169276               92.4518             
  5669    19     83800  2252/429

  5790    19     85600  4052/4292       94% 0.176718               91.6856             
  5793    19     85650  4102/4292       96% 0.280753               91.6658             
  5795    19     85700  4152/4292       97% 0.206078               91.6383             
  5798    19     85750  4202/4292       98% 0.298875               91.6195             
  5801    19     85800  4252/4292       99% 0.263462               91.5966             


モデルを読み込んでテストデータで評価

In [18]:
if True:
    path_model_eval = "results_300d/best_snapshot_devacc_76.92542166226377_devloss_0.8656771183013916__iter_21000_model.pt"
    model_eval = torch.load(path_model_eval,
                            map_location=lambda storage,
                            locatoin: storage.cuda(args.gpu))

    def evaluate(model, data_iter, data_size):
        log_template = '{:>8.6f}, {:8.6f}, {:12.4f}%'
        model.eval(); data_iter.init_epoch()

        # calculate accuracy on validation set
        n_correct, loss = 0, 0
        with torch.no_grad():
            for batch_idx, batch in enumerate(data_iter):
                answer = model(batch)
                n_correct += (torch.max(answer, 1)[1].view(batch.label.size()) == batch.label).sum().item()
                loss = criterion(answer, batch.label)
        acc = 100. * n_correct / data_size

        print(log_template.format(loss.item(), loss.item(), acc))

    evaluate(model_eval, test_iter, len(test))

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/content/drive/practice/colaboratory/180519_snli_lstm_with_w2v_by_pytorch/model.py:34: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
  outputs, (ht, ct) = self.rnn(inputs, (h0, c0))


0.639084, 0.639084,      76.5778%


google driveとの同期プロセスも完全にkillする

In [0]:
# !kill -9 -1